In [1]:
include("../src/MTJWAS.jl")

MTJWAS

## <font color="blue"> Data</font>

In [2]:
using DataFrames

In [3]:
DIR="data/"
phenofile=DIR*"phenotype.txt"
genofile =DIR*"genotype.txt"
pedfile  =DIR*"pedigree.txt";

### phenotypes

In [4]:
;cat $phenofile

Animal,BW,CW,age,sex
S1,100.0,10.0,8,M
D1,50.0,12.9,7,F
O1,150.0,13.0,3,M
O3,40.0,5.0,4,F


### genotypes

In [5]:
;cat $genofile

Animal,x1,x2,x3,x4,x5
S1,1.0,0.0,1.0,1.0,1.0
D1,2.0,0.0,2.0,2.0,1.0
O1,1.0,2.0,0.0,1.0,0.0
O3,0.0,0.0,2.0,1.0,1.0


### pedigree

In [6]:
;cat $pedfile

S1 0 0
D1 0 0
O1 S1 D1
O2 S1 D1
O3 S1 D1


### Genetic covariance matrix and residual covariance matrix

In [7]:
R      = [10.0 2.0 1.0
           2.0 1.0 0.5
           1.0 0.5 3.0]   

G      = [20.0 1.0 1.0
           1.0 1.0 0.5
           1.0 0.5 3.0] ;

## <font color="blue">1. mulitiple trait with fixed effects</font> 

In [8]:
using MTJWAS

In [9]:
data=readtable(phenofile)

,Animal,BW,CW,age,sex
1,S1,100.0,10.0,8,M
2,D1,50.0,12.9,7,F
3,O1,150.0,13.0,3,M
4,O3,40.0,5.0,4,F


In [10]:
data[:EMA] = @data([10.0,20.0,30.0,40.0]);

In [11]:
data

,Animal,BW,CW,age,sex,EMA
1,S1,100.0,10.0,8,M,10.0
2,D1,50.0,12.9,7,F,20.0
3,O1,150.0,13.0,3,M,30.0
4,O3,40.0,5.0,4,F,40.0


## <font color="blue">3. mulitiple traits with marker effects</font> 

#### set up model equations

In [12]:
model_equations = "BW  = intercept + age + sex;
                   EMA = intercept + age + sex;
                   CW  = intercept + age + sex";

In [13]:
model3    = buildModel(model_equations,R);

In [14]:
setAsCovariate(model3,"age")

In [15]:
addMarkers(model3,genofile,G,separator=',',header=true);

The delimiters in file data/genotype.txt is ,  .


In [16]:
out3=runMCMC(model3,data,chain_length=50000,printout_frequency=10000);

at sample: 10000
Residual covariance matrix: 
[36.58631026033691 7.275511463722335 3.882665437782223
 7.275511463722335 3.2365319636497163 0.9231137785644008
 3.882665437782223 0.9231137785644008 3.903941701626138]
Marker effects covariance matrix: 
[48.94674549307252 -7.729409802063201 9.70841304146252
 -7.729409802063201 2.3388358547069696 -1.664688447431086
 9.70841304146252 -1.664688447431086 3.3169019321895172]

at sample: 20000
Residual covariance matrix: 
[26.083369179295666 3.9995890494706243 3.474647116919625
 3.9995890494706243 2.2200461907187234 0.4876491084535151
 3.474647116919625 0.4876491084535151 3.8236133709870113]
Marker effects covariance matrix: 
[57.62052440800447 -9.273367025397356 11.456148680351479
 -9.273367025397356 2.5368909528087813 -1.9382295546506665
 11.456148680351479 -1.9382295546506665 3.694803638529671]

at sample: 30000
Residual covariance matrix: 
[21.59120032013124 3.0428470733667283 3.012308362571223
 3.0428470733667283 1.8471167413145644 0.389903

In [17]:
keys(out3)

Base.KeyIterator for a Dict{Any,Any} with 4 entries. Keys:
  "posterior mean of marker effects covariance matrix"
  "posterior mean of residual covaraince matrix"
  "posterior mean of marker effects"
  "posterior mean of location parameters"

### BayesC

In [18]:
model_equations = "BW  = intercept + age + sex;
                   EMA = intercept + age + sex;
                   CW  = intercept + age + sex";
model4    = buildModel(model_equations,R);
setAsCovariate(model4,"age")
addMarkers(model4,genofile,G,separator=',',header=true);

The delimiters in file data/genotype.txt is ,  .


In [19]:
out4=runMCMC(model4,data,Pi=[0.0 0.0 0.0],chain_length=50000,printout_frequency=10000);

marker effect varaince with π=[0.0 0.0 0.0] is positive definite??: true
at sample: 10000
Residual covariance matrix: 
[49.70953967623094 9.925945022873163 7.883589344585516
 9.925945022873163 2.879819152753454 1.735330780893492
 7.883589344585516 1.735330780893492 4.302250877706452]
Marker effects covariance matrix: 
[56.186720630008836 -9.15481137596869 11.502740785266955
 -9.15481137596869 2.404908552775645 -2.0363137583403654
 11.502740785266955 -2.0363137583403654 3.7757841407017954]

at sample: 20000
Residual covariance matrix: 
[29.534236240162986 5.837180206216203 4.440519859141298
 5.837180206216203 1.9208554674036948 1.0745121293347903
 4.440519859141298 1.0745121293347903 3.603814992365473]
Marker effects covariance matrix: 
[60.70196593682909 -9.706694061976165 12.469858758032741
 -9.706694061976165 2.4007676728430543 -2.1217303833287255
 12.469858758032741 -2.1217303833287255 3.9815708009027047]

at sample: 30000
Residual covariance matrix: 
[24.09844462455617 4.2258379816

In [20]:
keys(out4)

Base.KeyIterator for a Dict{Any,Any} with 4 entries. Keys:
  "posterior mean of marker effects covariance matrix"
  "posterior mean of residual covaraince matrix"
  "posterior mean of marker effects"
  "posterior mean of location parameters"

In [21]:
out4["posterior mean of location parameters"]

12x2 Array{Any,2}:
 "1:intercept: intercept"  -170.563   
 "1:age: age"                -6.00358 
 "1:sex: M"                 323.47    
 "1:sex: F"                 253.271   
 "2:intercept: intercept"  -107.297   
 "2:age: age"                -4.74908 
 "2:sex: M"                 154.33    
 "2:sex: F"                 162.403   
 "3:intercept: intercept"   -40.6538  
 "3:age: age"                 0.360657
 "3:sex: M"                  49.0259  
 "3:sex: F"                  48.6815  

In [22]:
out3["posterior mean of location parameters"]

12x2 Array{Any,2}:
 "1:intercept: intercept"   93.2021  
 "1:age: age"               -5.95938 
 "1:sex: M"                 58.8707  
 "1:sex: F"                -10.0171  
 "2:intercept: intercept"   99.0738  
 "2:age: age"               -4.78832 
 "2:sex: M"                -51.7976  
 "2:sex: F"                -43.7585  
 "3:intercept: intercept"  -96.1103  
 "3:age: age"                0.381298
 "3:sex: M"                104.268   
 "3:sex: F"                104.202   

In [23]:
out3["posterior mean of marker effects"]

3-element Array{Any,1}:
 5x2 Array{Any,2}:
 "x1"  10.8704 
 "x2"   6.12559
 "x3"  -3.79361
 "x4"   4.77846
 "x5"  -3.22866     
 5x2 Array{Any,2}:
 "x1"  -2.04765 
 "x2"  -1.18437 
 "x3"   0.668379
 "x4"  -0.955233
 "x5"   0.615743
 5x2 Array{Any,2}:
 "x1"   2.39571 
 "x2"   1.34923 
 "x3"  -0.85441 
 "x4"   1.0696  
 "x5"  -0.742362

In [24]:
out4["posterior mean of marker effects"]

3-element Array{Any,1}:
 5x2 Array{Any,2}:
 "x1"  10.9944 
 "x2"   6.21623
 "x3"  -3.24278
 "x4"   5.57142
 "x5"  -3.48161     
 5x2 Array{Any,2}:
 "x1"  -2.13388 
 "x2"  -1.16663 
 "x3"   0.599436
 "x4"  -1.06592 
 "x5"   0.648093
 5x2 Array{Any,2}:
 "x1"   2.4893  
 "x2"   1.38636 
 "x3"  -0.735363
 "x4"   1.26573 
 "x5"  -0.793239

In [25]:
X=readdlm(genofile,',',header=true)[1]

4x6 Array{Any,2}:
 "S1"  1.0  0.0  1.0  1.0  1.0
 "D1"  2.0  0.0  2.0  2.0  1.0
 "O1"  1.0  2.0  0.0  1.0  0.0
 "O3"  0.0  0.0  2.0  1.0  1.0

In [26]:
X=X[:,2:end]

4x5 Array{Any,2}:
 1.0  0.0  1.0  1.0  1.0
 2.0  0.0  2.0  2.0  1.0
 1.0  2.0  0.0  1.0  0.0
 0.0  0.0  2.0  1.0  1.0

In [27]:
a=X*map(Float64,out3["posterior mean of marker effects"][1][:,2])

4-element Array{Any,1}:
  8.62657
 20.4818 
 27.9    
 -6.03743

In [28]:
b=X*map(Float64,out4["posterior mean of marker effects"][1][:,2])

4-element Array{Any,1}:
  9.84146
 23.1645 
 28.9983 
 -4.39575

In [29]:
cor(a,b)

0.9988274651601142